In [1]:
import pip
import os
import sys
import re
#import preprocessor as p
import pandas as pd
import pip
import numpy as np
from nltk.stem.porter import PorterStemmer
from googletrans import Translator

In [2]:
use_stemmer = False
porter_stemmer = PorterStemmer()
test_file = False

translator = Translator()

In [3]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

In [4]:
def translate(sent):
    trans = translator.translate(sent).text
    return trans


def preprocess_word(word):
    # Remove punctuation
    word = word.strip('\'"?!,.():;')
    word = re.sub(r'(.)\1+', r'\1\1', word)
    # Remove - & '
    word = re.sub(r'(-|\')', '', word)
    return word


def is_valid_word(word):
    # Check if word begins with an alphabet
    return (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None)


def handle_emojis(tweet):
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', tweet)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    tweet = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' EMO_POS ', tweet)
    # Love -- <3, :*
    tweet = re.sub(r'(<3|:\*)', ' EMO_POS ', tweet)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    tweet = re.sub(r'(;-?\)|;-?D|\(-?;)', ' EMO_POS ', tweet)
    # Sad -- :-(, : (, :(, ):, )-:
    tweet = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMO_NEG ', tweet)
    # Cry -- :,(, :'(, :"(
    tweet = re.sub(r'(:,\(|:\'\(|:"\()', ' EMO_NEG ', tweet)
    # remove other emojis
    tweet = emoji_pattern.sub(r'', tweet)
    #print(tweet)
    
    return tweet


def preprocess_tweet(tweet):
    processed_tweet = []
    tweet = handle_emojis(str(tweet))
    tweet = str(translate(tweet))
    tweet = tweet.lower()
    tweet = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' URL ', tweet)
    tweet = re.sub(r'@[\S]+', 'USER_MENTION', tweet)
    tweet = re.sub(r'#(\S+)', r' \1 ', tweet)
    tweet = re.sub(r'\brt\b', '', tweet)
    tweet = re.sub(r'\.{2,}', ' ', tweet)
    tweet = tweet.strip(' "\'')
    
    tweet = re.sub(r'\s+', ' ', tweet)
    words = tweet.split()

    for word in words:
        word = preprocess_word(word)
        if is_valid_word(word):
            if use_stemmer:
                word = str(porter_stemmer.stem(word))
            processed_tweet.append(word)

    return ' '.join(processed_tweet)


def preprocess(raw_data):
    processed_file = 'processed_file_only_tweets.csv'
    processed_file_open = open(processed_file,'a+')
    
    with open(raw_data,'r') as csv:
        lines = csv.readlines()
        total = len(lines)
        #print(total)
        count=0
        for i,j in enumerate(lines):
            tweet_id = j[:j.find(',')]
            j = j[1+j.find(','):]
            name = j[:j.find(','):]
            j = j[1+j.find(','):]
            #print(name)
            handle = j[:name.find(',')]
            j = j = j[1+j.find(','):]
            #print(handle)
            tweet = j[1+handle.find(','):]
            #print(tweet)
            processed_tweet = preprocess_tweet(tweet)
            #print(processed_tweet)
            #print("-----------")
#             processed_file_open.write(tweet_id)
#             processed_file_open.write(',')
#             processed_file_open.write(name)
#             processed_file_open.write(',')
#             processed_file_open.write(handle)
#             processed_file_open.write(',')
            processed_file_open.write(processed_tweet)
            processed_file_open.write("\n")
#             if(count>4):
#                 break
#             count += 1
    processed_file_open.close()


In [5]:
raw_data = pd.read_csv("Political-bias-detection/Dataset/No_RTs_quotes_added.csv")
raw_data.head()

Tweet_id        User_Name User_Screen_Name  \
0  983961802395537408  Arvind Kejriwal   ArvindKejriwal   
1  983709192673546240  Arvind Kejriwal   ArvindKejriwal   
2  983581950970560512  Arvind Kejriwal   ArvindKejriwal   
3  982878153377972224  Arvind Kejriwal   ArvindKejriwal   
4  982550408617963521  Arvind Kejriwal   ArvindKejriwal   

                                                Text  
0  \nBJP’s dirty politics exposed. Inside the mee...  
1  Now that’s really cute .... just one day fast....  
2  \nबात नीयत की है। क्या नीयत काम की थी या केवल ...  
3  \nTDP MPs taken to Tughlak Road Police Stn for...  
4  \nदेश में राजनीति इन मुद्दों पर होनी चाहिए। न ...

In [6]:
processed_file = 'processed_file_only_tweets.csv'
processed_file_open = open(processed_file,'a+')
count=0
for i in raw_data['Text']:
#     if(count>7):
#         break
    #print(i)
    #print(preprocess_tweet(i))
    processed_file_open.write(preprocess_tweet(i))
    processed_file_open.write('\n')
#     print("----------")
#     count += 1
    

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [11]:
#tt = pd.read_csv('processed_file_only_tweets.csv',header = None)
#tt.head()
test = str(translate(raw_data['Text'][2177]))
print(test)

@mebinod Thanks


In [30]:
pd.DataFrame.to_csv(raw_data,"No_newLines1.csv")

In [31]:
tt = pd.read_csv('No_newLines1.csv')
tt.head()
#preprocess("No_newLines.csv")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x92 in position 5: invalid start byte

In [ ]:
# print("---")
# data = pd.read_csv("Political-bias-detection/Dataset/No_RTs_quotes_added.csv")

In [5]:
data.head()
# data = data.replace('\n',' ', regex=True)
# pd.DataFrame.to_csv(data,'No_RTs_quotes_added_1.csv')
# # count = 0
# # for i in data.Text:
# #     print(i)
# #     re.sub('\n',' ',i)
# #     print("----------")
# #     if(count>4):
# #         break
# #     count += 1
# # #re.sub('\n',' ',data.Text)
col_a = data['Tweet_id']
#print(col_a)
df1 = pd.DataFrame(data['Tweet_id'], data['Tweet_id'],data['Tweet_id'])
df1.head()

NameError: name 'data' is not defined

In [ ]:
preprocess("Political-bias-detection/Dataset/No_RTs_quotes_added.csv")

In [7]:
#preprocess_tweet("s in various Delhi Govt Schemeshttps://youtu.be/URHFraquUCg  ,ts a question" )
preprocess_tweet("बाबा साहेब भीमराव अम्बेडकर जी के महापरिनिर्वाण दिवस पर कोटि-कोटि नमन🙏" ) 
# tweets = pd.read_csv('processed_file_only_tweets.csv',header = None)

'saheb bhimrao ambedkar celebrated mahaparinirvanas day at kotikoti'

In [ ]:
data.Text = tweets

In [ ]:
data.head()

In [ ]:
# pd.DataFrame.to_csv(data,"Intermediate_dataset.csv",index = False)

In [ ]:
# to_translate = pd.read_csv("Political-bias-detection/Dataset/No_RTs_quotes_added.csv")
# to_translate.head()

In [ ]:
# translated_data = open('translated_data.csv','a+')

In [ ]:
# print(len(to_translate.Text))
# for i in range(len(to_translate.Text)):
#     temp = emoji_pattern.sub(r'',str(to_translate.Text[i]))
#     translated_data.write(translator.translate(temp).text)
#     print(i)
#     translated_data.write('\n')

In [ ]:
# to_translate.Text[201]
# translator.translate('बाबा साहेब भीमराव अम्बेडकर जी के महापरिनिर्वाण दिवस पर कोटि-कोटि नमन').text
# print(emoji_pattern.sub(r'', 'बाबा साहेब भीमराव अम्बेडकर जी के महापरिनिर्वाण दिवस पर कोटि-कोटि नमन🙏'))